In [1]:
! pip install -q "gdown==5.2.0"

In [2]:
!gdown --id 1xsXrlyyQm_SXawu_J_foKZFRApC3NCpS

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1xsXrlyyQm_SXawu_J_foKZFRApC3NCpS
To: /content/pdf.zip
100% 5.17M/5.17M [00:00<00:00, 146MB/s]


In [3]:
!unzip pdf.zip

Archive:  pdf.zip
  inflating: pdf/LLaVA.pdf           


In [4]:
!pip install -q sentence-transformers
!pip install -q pymupdf
!pip install -q numpy
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 90.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.2 MB/s eta 0:00:00


In [5]:
from google.colab import userdata
from openai import OpenAI

In [6]:
client = OpenAI(api_key=userdata.get('openai_api'))

In [7]:
import fitz  # PyMuPDF

def extract_paragraphs_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    paragraphs = []

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text("text")
        page_paragraphs = text.split('\n\n')
        paragraphs.extend(page_paragraphs)

    return paragraphs

pdf_path = '/content/pdf/LLaVA.pdf'
paragraphs = extract_paragraphs_from_pdf(pdf_path)


In [8]:
import textwrap

In [ ]:
for i, p in enumerate(paragraphs):
  wrapped_text = textwrap.fill(p, width=100)

  print("-----------------------------------------------------------------")
  print(wrapped_text)
  print("-----------------------------------------------------------------")

# Embedding model

In [10]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("Alibaba-NLP/gte-base-en-v1.5", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/71.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [13]:
batch_size = 2  # Adjust batch size based on available memory
docs_embed = []
for i in range(0, len(paragraphs), batch_size):
    batch = paragraphs[i:i+batch_size]
    embeddings = model.encode(batch, normalize_embeddings=True)
    docs_embed.extend(embeddings)


In [32]:
import numpy as np

# Assuming docs_embed is your list of embeddings
docs_embed = np.array(docs_embed)
print(docs_embed.shape)


(25, 768)


In [15]:
docs_embed[0]

array([-3.18465359e-03, -3.55901495e-02, -1.57167576e-02,  5.68832830e-03,
        7.17788527e-04,  1.10342549e-02,  3.37103233e-02, -2.50817556e-02,
        2.03014631e-03, -5.54723339e-03,  2.13214718e-02,  1.67628210e-02,
        6.92339092e-02, -1.22790877e-02,  1.79569349e-02, -2.74467357e-02,
       -3.99930999e-02, -2.66702585e-02,  2.90033175e-03, -3.50243524e-02,
       -4.06241836e-03,  1.46121613e-03,  2.31142491e-02,  5.66453533e-03,
       -2.02646740e-02, -2.04069596e-02,  2.91288737e-03,  2.67139561e-02,
        4.55984958e-02, -2.16249861e-02, -4.17601457e-03, -2.10269410e-02,
       -1.90915465e-02, -1.46815730e-02, -1.57586280e-02,  2.86121685e-02,
       -6.22310191e-02,  5.56185395e-02, -7.97222778e-02,  6.66238666e-02,
       -9.49930474e-02,  7.28606666e-03,  1.13515249e-02, -1.66792702e-02,
       -9.67721455e-03,  7.71347955e-02, -5.45474626e-02, -2.09149509e-03,
        9.67388600e-03, -2.24569347e-02,  1.53141096e-02, -5.59281884e-03,
        1.77087933e-02,  

# Embed the Query

In [18]:
query = "What are the LLMs trained on image-text pairs?"
query_embed = model.encode(query, normalize_embeddings=True)

In [19]:
query_embed.shape

(768,)

# Find the Closest Paragraphs to the Query

In [20]:
import numpy as np
similarities = np.dot(docs_embed, query_embed.T)

In [21]:
similarities.shape

(25,)

In [22]:
similarities

array([0.60228133, 0.644054  , 0.5384798 , 0.60089993, 0.5557829 ,
       0.53296244, 0.554567  , 0.51590645, 0.570125  , 0.5707488 ,
       0.5230238 , 0.59259653, 0.5915273 , 0.5372057 , 0.5857537 ,
       0.28285468, 0.26511437, 0.34307188, 0.38001484, 0.525018  ,
       0.49184284, 0.49078333, 0.3669213 , 0.32335132, 0.21852395],
      dtype=float32)

In [23]:
top_3_idx = np.argsort(similarities, axis=0)[-3:][::-1].tolist()

In [24]:
top_3_idx

[1, 0, 3]

In [25]:
most_similar_documents = [paragraphs[idx] for idx in top_3_idx]

In [26]:
CONTEXT = ""
for i, p in enumerate(most_similar_documents):
  wrapped_text = textwrap.fill(p, width=100)

  print("-----------------------------------------------------------------")
  print(wrapped_text)
  print("-----------------------------------------------------------------")
  CONTEXT += wrapped_text + "\n\n"

-----------------------------------------------------------------
utilize various machine-generated high-quality instruction-following samples to improve the LLM’s
alignment ability, reporting impressive performance compared with proprietary LLMs. Importantly,
this line of work is text-only. In this paper, we present visual instruction-tuning, the first
attempt to extend instruction-tuning to the language-image multimodal space, to pave the way towards
building a general-purpose visual assistant. In particular, our paper makes the following
contributions: • Multimodal instruction-following data. One key challenge is the lack of vision-
language instruction-following data. We present a data reformation perspective and pipeline to
convert image-text pairs into an appropriate instruction-following format, using ChatGPT/GPT-4. •
Large multimodal models. We develop a large multimodal model (LMM), by connecting the open-set
visual encoder of CLIP [40] with the language decoder Vicuna [9], an

In [27]:
query = "What are the LLMs trained on image-text pairs?"

In [28]:
prompt = f"""
use the following CONTEXT to answer the QUESTION at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

CONTEXT: {CONTEXT}
QUESTION: {query}

"""

In [29]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "user", "content": prompt},
  ]
)

In [30]:
print(response.choices[0].message.content)

The LLMs trained on image-text pairs mentioned in the context are:

- Flamingo
- BLIP-2
- FROMAGe
- KOSMOS-1
- PaLM-E
- OpenFlamingo
- LLaMA-Adapter
